In [30]:
# импортируем необходимые библиотеки, классы и функции
import numpy as np  
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [31]:
# создаем игрушечный массив признаков
X_toy = np.array([[0.1, 0.2, 0.3], 
                  [0.7, 0.5, 0.2],
                  [0.2, 0.4, 1.4],
                  [0.4, 0.1, 0.5]])

# создаем игрушечный массив значений
# зависимой переменной
y_toy = np.array([0, 1, 0, 0])

In [32]:
# ?
from random import randint
randint(0, len(X_toy) - 1)

2

In [33]:
# создаем вектор весов, инициализируем веса очень 
# небольшими положительными значениями, близкими к 0
theta = np.array([0.01, 0.012, 0.015])
# theta = np.array([0.0, 0.0, 0.0])

In [34]:
### квазиньютоновская инициализация
# создаем начальное приближение обратного гессиана
N = len(theta)
I = np.eye(N, dtype=int)
H = I

In [35]:
# пишем функцию сигмоид-преобразования
def sigmoid(z):
    z[z > 100] = 100
    z[z < -100] = -100
    return 1 / (1 + np.exp(-z))

# пишем функцию, вычисляющую 
# логистическую функцию потерь
def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

In [36]:
number_iter = 100
for i in range(1, number_iter):
    # вычисляем скалярное произведение 
    # матрицы признаков и вектора весов
    z = np.dot(X_toy, theta)
    # к скалярному произведению применяем сигмоид-преобразование,
    # получаем вероятности положительного класса
    h = sigmoid(z)
    # получаем вектор градиента, для этого делим скалярное произведение 
    # транспонированной матрицы признаков и вектора разностей между 
    # вероятностями положительного класса и фактическими 
    # значениями зависимой переменной на количество наблюдений
    gradient = np.dot(X_toy.T, (h - y_toy)) / y_toy.size
    
    ### вычисляем направление шага
    p = - np.dot(H, gradient)
    
    ### вычисляем длину шага перебором
    theta_tmp = theta
    loss_min = float("inf")
    number_iter_alpha = 100
    for alpha_i in range(1, number_iter_alpha, 1):
        h = sigmoid(X_toy.dot(theta_tmp))
        loss_i = loss(h, y_toy)
        if loss_i < loss_min:
            loss_min = loss_i
            alpha = alpha_i/100
        theta_tmp = theta + alpha_i/100 * p
    
    ### вычисляем аргументы приближения обратного гессиана и само приближение
    ### https://habr.com/ru/post/333356/
    theta_delta = (theta + alpha * p) - theta
    gradient_k1 = np.dot(X_toy.T, (sigmoid(X_toy.dot(theta + alpha * p)) - y_toy)) / y_toy.size
    gradient_k = np.dot(X_toy.T, (sigmoid(X_toy.dot(theta)) - y_toy)) / y_toy.size
    grad_delta = gradient_k1 - gradient_k
    ro = 1.0 / (np.dot(grad_delta, theta_delta))
    A1 = I - ro * theta_delta[:, np.newaxis] * grad_delta[np.newaxis, :]
    A2 = I - ro * grad_delta[:, np.newaxis] * theta_delta[np.newaxis, :]
    H = np.dot(A1, np.dot(H, A2)) + (ro * theta_delta[:, np.newaxis] * theta_delta[np.newaxis, :])
    ### выполняем обновление весов
    theta = theta + alpha * p
    ### выводим промежуточные результаты
    print(i, alpha, loss(h, y_toy), 'коэффициенты', theta)

1 0.99 0.6385552059298325 коэффициенты [ 0.00859981 -0.01410934 -0.23564556]
2 0.99 0.4522128749233257 коэффициенты [ 0.07063818 -0.10413247 -1.65059233]
3 0.99 0.40795573708183114 коэффициенты [ 0.17778785 -0.08906256 -2.34307169]
4 0.99 0.3487669493752265 коэффициенты [ 0.47791524  0.00845571 -3.66776533]
5 0.99 0.28127142409095035 коэффициенты [ 1.0556488   0.23726199 -5.71939956]
6 0.99 0.19712424705191556 коэффициенты [ 2.18853191  0.7206634  -9.29634893]
7 0.99 0.1187207227522745 коэффициенты [  3.87012153   1.46607751 -14.26186498]
8 0.99 0.05800475167571265 коэффициенты [  6.13803719   2.48629017 -20.78673509]
9 0.99 0.02693224932127892 коэффициенты [  8.43387288   3.52295664 -27.34929577]
10 0.99 0.012992176994001617 коэффициенты [ 10.54614726   4.47742395 -33.37991348]
11 0.99 0.006386068403480965 коэффициенты [ 12.57404771   5.39389986 -39.16825291]
12 0.99 0.0031758694957892512 коэффициенты [ 14.55428514   6.28886626 -44.8202287 ]
13 0.99 0.0015874202761297302 коэффициенты 

C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in multiply
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in multiply
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less
  after removing the cwd from sys.path.
